# Feature extraction

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
import glob

jab_color_files = [file for file in glob.glob('./numpy_data_arrays/jab/*color.npz')]
cross_color_files = [file for file in glob.glob('./numpy_data_arrays/cross/*color.npz')]
left_hook_color_files = [file for file in glob.glob('./numpy_data_arrays/left_hook/*color.npz')]
right_hook_color_files = [file for file in glob.glob('./numpy_data_arrays/right_hook/*color.npz')]
random_color_files = [file for file in glob.glob('./numpy_data_arrays/random/*color.npz')]

In [ ]:
# Extract 6 frames evenly spread out
def extract_frames(frames):
    extracted_frames = []
    for i in range(1, 7):
        frame_to_extract = (i * (frames.shape[0]) // 6 ) - 1
        extracted_frames.append(frames[frame_to_extract])
    return np.array(extracted_frames)

In [149]:
load_data = np.load(right_hook_color_files[5])
color_data = load_data["arr_0"]
frames = extract_frames(color_data)


for i, frame in enumerate(frames):
    cv2.imshow(f'Frame {i}', frame)

cv2.waitKey(0)
cv2.destroyAllWindows()


(6, 383, 383, 4)

## MediaPipe

In [ ]:
def get_landmarks(frames):
    landmarks_coords = []

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        for frame in frames:
            image = cv2.cvtColor(frame, cv2.COLOR_BGRA2RGB)
            image.flags.writeable = False

            pose_results = pose.process(image)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                landmarks = pose_results.pose_landmarks.landmark
                # print (landmarks)
                landmarks_coords.append([[landmark.x, landmark.y] for landmark in landmarks])

            except:
                pass
        return np.array(landmarks_coords).ravel()

In [ ]:
X = []
y = []

for file in jab_color_files:
    load_data = np.load(file)
    color_data = load_data['arr_0']
    frames = extract_frames(color_data)
    landmarks = get_landmarks(frames)
    X.append(landmarks)
    y.append('jab')

for file in cross_color_files:
    load_data = np.load(file)
    color_data = load_data['arr_0']
    frames = extract_frames(color_data)
    landmarks = get_landmarks(frames)
    X.append(landmarks)
    y.append('cross')

for file in left_hook_color_files:
    load_data = np.load(file)
    color_data = load_data['arr_0']
    frames = extract_frames(color_data)
    landmarks = get_landmarks(frames)
    X.append(landmarks)
    y.append('left_hook')

for file in right_hook_color_files:
    load_data = np.load(file)
    color_data = load_data['arr_0']
    frames = extract_frames(color_data)
    landmarks = get_landmarks(frames)
    X.append(landmarks)
    y.append('right_hook')

for file in random_color_files:
    load_data = np.load(file)
    color_data = load_data['arr_0']
    frames = extract_frames(color_data)
    landmarks = get_landmarks(frames)
    X.append(landmarks)
    y.append('random')

# Classification

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

X = np.array(X)
y = np.array(y)
X, y = shuffle(X, y, random_state=0)

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(y)
class_labels = label_encoder.classes_

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict

def run_kfolds(model, x, y, splits):
    kf = KFold(n_splits=splits, shuffle=True)
    scores = cross_val_score(model, x, y, cv=kf, scoring="accuracy")
    predict = cross_val_predict(model, x, y, cv=kf)
    return scores, predict

## SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
from matplotlib import pyplot as plt

SVM = SVC(kernel='linear')

SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
svm_report = classification_report(y_test, y_pred)

# Display the evaluation metrics
print("Accuracy:", accuracy)
print("SVM Classification Report:")
print(svm_report)

cm = confusion_matrix(y_test, y_pred)
# Get the class labels from LabelEncoder
# class_labels = label_encoder.classes_

# Display the confusion matrix as a heatmap
plt.figure(figsize=(3,3))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

### Cross validation

In [ ]:
SVM = SVC(kernel='linear')

scores, predicts = run_kfolds(SVM, X, y, 5)

# Calculate evaluation metrics
accuracy = accuracy_score(y, predicts)
svm_report = classification_report(y, predicts)

# Display the evaluation metrics
print("Accuracy:", accuracy)
print("SVM Cross-Validation Classification Report:")
print(svm_report)

cm = confusion_matrix(y, predicts)

# Display the confusion matrix as a heatmap
plt.figure(figsize=(3, 3))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# landmarks[0].x

In [ ]:
# load_data = np.load(jab_color_files[1])
# color_data = load_data['arr_0']

# with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#     for frame in color_data:
#         image = cv2.cvtColor(frame, cv2.COLOR_BGRA2RGB)
#         image.flags.writeable = False
#         cv2.imshow("IMAGE DETECTIONS", image)

#         pose_results = pose.process(image)

#         image.flags.writeable = True
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#         mp_drawing.draw_landmarks(image, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
#                                   mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
#                                   mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
#                                   )
#         cv2.imshow("MEDIAPIPE DETECTIONS", image)
        
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()


In [ ]:
# with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#     for frame in color_data:
#         image = cv2.cvtColor(frame, cv2.COLOR_BGRA2RGB)
#         image.flags.writeable = False

#         pose_results = pose.process(image)

#         image.flags.writeable = True
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#         try:
#             landmarks = pose_results.pose_landmarks.landmark
#             print (landmarks)
#         except:
#             pass

#         mp_drawing.draw_landmarks(image, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
#                                   mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
#                                   mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
#                                   )
#         cv2.imshow("MEDIAPIPE DETECTIONS", image)
        
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()


In [ ]:
# for landmark in mp_pose.PoseLandmark:
#     print((landmark))

[(e, e.value) for e in mp_pose.PoseLandmark]

Iterations vs Accuracy

## Naive Bayes

### Cross validation

In [ ]:
from sklearn.naive_bayes import GaussianNB

NB = GaussianNB()

scores, predicts = run_kfolds(NB, X, y, 5)

# Calculate evaluation metrics
accuracy = accuracy_score(y, predicts)
nb_report = classification_report(y, predicts)

# Display the evaluation metrics
print("Accuracy:", accuracy)
print("NB Cross-Validation Classification Report:")
print(nb_report)

cm = confusion_matrix(y, predicts)

# Display the confusion matrix as a heatmap
plt.figure(figsize=(3, 3))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

## Random Forest

### Cross validation

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)

# rf.fit(X_train, y_train)

scores, predicts = run_kfolds(rf, X, y, 5)

# Calculate evaluation metrics
accuracy = accuracy_score(y, predicts)
rf_report = classification_report(y, predicts)

# Display the evaluation metrics
print("Accuracy:", accuracy)
print("Random Forest Cross-Validation Classification Report:")
print(rf_report)

cm = confusion_matrix(y, predicts)

# Display the confusion matrix as a heatmap
plt.figure(figsize=(3, 3))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
# y_pred = clf.predict(X_test)

# # Calculate evaluation metrics
# accuracy = accuracy_score(y_test, y_pred)
# mlp_report = classification_report(y_test, y_pred)

# # Display the evaluation metrics
# print("Accuracy:", accuracy)
# print("mlp Classification Report:")
# print(mlp_report)

# cm = confusion_matrix(y_test, y_pred)
# # Get the class labels from LabelEncoder
# # class_labels = label_encoder.classes_

# # Display the confusion matrix as a heatmap
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
# plt.xlabel('Predicted')
# plt.ylabel('True')
# plt.show()

## Multi-Layer Perceptron

In [ ]:
#import the classifier
from sklearn.neural_network import MLPClassifier

# initialize the classifier
#MLP = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
MLP = MLPClassifier(max_iter=150)

MLP.fit(X_train, y_train)
y_pred = MLP.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
mlp_report = classification_report(y_test, y_pred)

# Display the evaluation metrics
print("Accuracy:", accuracy)
print("mlp Classification Report:")
print(mlp_report)

cm = confusion_matrix(y_test, y_pred)
# Get the class labels from LabelEncoder
# class_labels = label_encoder.classes_

# Display the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

### Cross validation

In [ ]:
#import the classifier
from sklearn.neural_network import MLPClassifier

# initialize the classifier
#MLP = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
MLP = MLPClassifier(max_iter=800)
scores, predicts = run_kfolds(MLP, X, y, 5)

# Calculate evaluation metrics
accuracy = accuracy_score(y, predicts)
mlp_report = classification_report(y, predicts)

# Display the evaluation metrics
print("Accuracy:", accuracy)
print("MLP Cross-Validation Classification Report:")
print(mlp_report)

cm = confusion_matrix(y, predicts)

# Display the confusion matrix as a heatmap
plt.figure(figsize=(3, 3))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
